In [ ]:
##################
# Classification #
##################

In [424]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn import svm, linear_model, naive_bayes, neural_network, neighbors, ensemble
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
import random, math
import numpy as np
import scipy.sparse as sp
from datetime import datetime
from collections import Counter

In [240]:
with open('mxl-list.txt', 'r') as f:
    dataset = [piece.strip() for piece in f.readlines()]
#     alkan_data = [f for f in dataset if f.replace('-', '_').split('_')[0] == 'alkan']
    bach_data = [f for f in dataset if f.replace('-', '_').split('_')[0] == 'bach']
    beethoven_data = [f for f in dataset if f.replace('-', '_').split('_')[0] == 'beethoven']
#     chopin_data = [f for f in dataset if f.replace('-', '_').split('_')[0] == 'chopin']
#     dandrieu_data = [f for f in dataset if f.replace('-', '_').split('_')[0] == 'dandrieu']
    debussy_data = [f for f in dataset if f.replace('-', '_').split('_')[0] == 'debussy']
#     handel_data = [f for f in dataset if f.replace('-', '_').split('_')[0] == 'handel']
#     haydn_data = [f for f in dataset if f.replace('-', '_').split('_')[0] == 'haydn']
#     liszt_data = [f for f in dataset if f.replace('-', '_').split('_')[0] == 'liszt']
#     mozart_data = [f for f in dataset if f.replace('-', '_').split('_')[0] == 'mozart']
#     pachelbel_data = [f for f in dataset if f.replace('-', '_').split('_')[0] == 'pachelbel']
    scarlatti_data = [f for f in dataset if f.replace('-', '_').split('_')[0] == 'scarlatti']
#     schubert_data = [f for f in dataset if f.replace('-', '_').split('_')[0] == 'schubert']
#     tchaikovsky_data = [f for f in dataset if f.replace('-', '_').split('_')[0] == 'tchaikovsky']
    victoria_data = [f for f in dataset if f.replace('-', '_').split('_')[0] == 'victoria']

In [248]:
# with open('alkan-chordsequence.txt', 'r') as f:
#     ALKAN = [' '.join(piece.strip('[]\n').split(', ')) for piece in f.readlines()]
#     ALKAN = [(ALKAN[i], alkan_data[i]) for i in range(len(ALKAN))]
with open('bach-chordsequence.txt', 'r') as f:
    BACH = [' '.join(piece.strip('[]\n').split(', ')) for piece in f.readlines()]
    BACH = [(BACH[i], bach_data[i]) for i in range(len(BACH))]
with open('beethoven-chordsequence.txt', 'r') as f:
    BEETHOVEN = [' '.join(piece.strip('[]\n').split(', ')) for piece in f.readlines()]
    BEETHOVEN = [(BEETHOVEN[i], beethoven_data[i]) for i in range(len(BEETHOVEN))]
# with open('chopin-chordsequence.txt', 'r') as f:
#     CHOPIN = [' '.join(piece.strip('[]\n').split(', ')) for piece in f.readlines()]
#     CHOPIN = [(CHOPIN[i], chopin_data[i]) for i in range(len(CHOPIN))]
# with open('dandrieu-chordsequence.txt', 'r') as f:
#     DANDRIEU = [' '.join(piece.strip('[]\n').split(', ')) for piece in f.readlines()]
#     DANDRIEU = [(DANDRIEU[i], dandrieu_data[i]) for i in range(len(DANDRIEU))]
with open('debussy-chordsequence.txt', 'r') as f:
    DEBUSSY = [' '.join(piece.strip('[]\n').split(', ')) for piece in f.readlines()]
    DEBUSSY = [(DEBUSSY[i], debussy_data[i]) for i in range(len(DEBUSSY))]
# with open('handel-chordsequence.txt', 'r') as f:
#     HANDEL = [' '.join(piece.strip('[]\n').split(', ')) for piece in f.readlines()]
#     HANDEL = [(HANDEL[i], handel_data[i]) for i in range(len(HANDEL))]
# with open('haydn-chordsequence.txt', 'r') as f:
#     HAYDN = [' '.join(piece.strip('[]\n').split(', ')) for piece in f.readlines()]
#     HAYDN = [(HAYDN[i], haydn_data[i]) for i in range(len(HAYDN))]
# with open('liszt-chordsequence.txt', 'r') as f:
#     LISZT = [' '.join(piece.strip('[]\n').split(', ')) for piece in f.readlines()]
#     LISZT = [(LISZT[i], liszt_data[i]) for i in range(len(LISZT))]
# with open('mozart-chordsequence.txt', 'r') as f:
#     MOZART = [' '.join(piece.strip('[]\n').split(', ')) for piece in f.readlines()]
#     MOZART = [(MOZART[i], mozart_data[i]) for i in range(len(MOZART))]
# with open('pachelbel-chordsequence.txt', 'r') as f:
#     PACHELBEL = [' '.join(piece.strip('[]\n').split(', ')) for piece in f.readlines()]
#     PACHELBEL = [(PACHELBEL[i], pachelbel_data[i]) for i in range(len(PACHELBEL))]
with open('scarlatti-chordsequence.txt', 'r') as f:
    SCARLATTI = [' '.join(piece.strip('[]\n').split(', ')) for piece in f.readlines()]
    SCARLATTI = [(SCARLATTI[i], scarlatti_data[i]) for i in range(len(SCARLATTI))]
# with open('schubert-chordsequence.txt', 'r') as f:
#     SCHUBERT = [' '.join(piece.strip('[]\n').split(', ')) for piece in f.readlines()]
#     SCHUBERT = [(SCHUBERT[i], schubert_data[i]) for i in range(len(SCHUBERT))]
# with open('tchaikovsky-chordsequence.txt', 'r') as f:
#     TCHAIKOVSKY = [' '.join(piece.strip('[]\n').split(', ')) for piece in f.readlines()]
#     TCHAIKOVSKY = [(TCHAIKOVSKY[i], tchaikovsky_data[i]) for i in range(len(TCHAIKOVSKY))]
with open('victoria-chordsequence.txt', 'r') as f:
    VICTORIA = [' '.join(piece.strip('[]\n').split(', ')) for piece in f.readlines()]
    VICTORIA = [(VICTORIA[i], victoria_data[i]) for i in range(len(VICTORIA))]

In [250]:
def find_ngrams(input_list, N=4):
    return [' '.join(input_list[i:i+N]) for i in range(len(input_list)-N+1)]

def ngrams_by_composer(composer): 
    for i in range(1,5):
        ngrams = []
        for piece in composer:
            ngrams += find_ngrams(piece[0].split(' '), i)
        print(len(ngrams), '{}-grams total;'.format(str(i)), len(set(ngrams)), 'unique')
    print('-')

def show_ngrams(composer_data, composer_name):
    print(composer_name, ':', len(composer_data), 'pieces')
    ngrams_by_composer(composer_data)

In [264]:
# show_ngrams(ALKAN,'alkan')
show_ngrams(BACH,'bach')
show_ngrams(BEETHOVEN,'beethoven')
# show_ngrams(CHOPIN,'chopin')
# show_ngrams(DANDRIEU,'dandrieu')
show_ngrams(DEBUSSY,'debussy')
# show_ngrams(HANDEL,'handel')
# show_ngrams(HAYDN,'haydn')
# show_ngrams(LISZT,'liszt')
# show_ngrams(MOZART,'mozart')
# show_ngrams(PACHELBEL,'pachelbel')
show_ngrams(SCARLATTI,'scarlatti')
# show_ngrams(SCHUBERT,'schubert')
# show_ngrams(TCHAIKOVSKY,'tchaikovsky')
show_ngrams(VICTORIA, 'victoria')
# show_ngrams(ALKAN+BACH+BEETHOVEN+CHOPIN+DANDRIEU+DEBUSSY+HANDEL+HAYDN+LISZT+MOZART+PACHELBEL+SCARLATTI+SCHUBERT+TCHAIKOVSKY+VICTORIA, 'all composers')
show_ngrams(BACH+BEETHOVEN+DEBUSSY+SCARLATTI+VICTORIA, 'all composers')

bach : 2245 pieces
766531 1-grams total; 179 unique
764286 2-grams total; 15783 unique
762041 3-grams total; 145459 unique
759796 4-grams total; 341141 unique
-
beethoven : 667 pieces
631995 1-grams total; 173 unique
631328 2-grams total; 16657 unique
630661 3-grams total; 132964 unique
629994 4-grams total; 265577 unique
-
debussy : 199 pieces
82362 1-grams total; 172 unique
82163 2-grams total; 10797 unique
81964 3-grams total; 38978 unique
81765 4-grams total; 55635 unique
-
scarlatti : 598 pieces
167106 1-grams total; 173 unique
166508 2-grams total; 10878 unique
165910 3-grams total; 57871 unique
165312 4-grams total; 103655 unique
-
victoria : 333 pieces
113980 1-grams total; 146 unique
113647 2-grams total; 4204 unique
113314 3-grams total; 22597 unique
112981 4-grams total; 50087 unique
-
all composers : 4042 pieces
1761974 1-grams total; 182 unique
1757932 2-grams total; 21648 unique
1753890 3-grams total; 274026 unique
1749848 4-grams total; 712690 unique
-


In [322]:
def build_Xy(composers, size=1):
    if size >= 1: # use every rows
        indices = [range(len(composer)) for composer in composers]
    else:
        indices = [random.sample(range(len(composer)), math.ceil(size*len(composer))) for composer in composers]

    y = []
    for i in range(len(composers)):
        y += [i for n in range(len(indices[i]))]
    
    X = []
    for i in range(len(composers)):
        X += [composers[i][j] for j in indices[i]]
    
    return X, np.array(y, dtype='int16')

In [384]:
def crossvalidate(X_tuple, y, classifiers, vectorizer, NGRAMRANGE, K=10):    
    for clf in classifiers:
        clf.cm_sum = np.zeros([len(set(y)),len(set(y))], dtype='int16')
        clf.accuracies, clf.fones, clf.misclassified, clf.runningtime = [], [], [], []
        clf.fones_micro, clf.fones_macro = [], []
        clf.name = str(clf).split('(')[0]

    X = np.array([piece[0] for piece in X_tuple])
    filenames = np.array([piece[1] for piece in X_tuple])
    kf = KFold(n_splits=K, shuffle=True)
    for train_index, test_index in kf.split(y):
        X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
        
        vct = vectorizer.set_params(lowercase=False, token_pattern=u"(?u)\\b\\w+\\b", ngram_range=NGRAMRANGE)
        X_train_tfidf = vct.fit_transform(X_train)
#         X_test_tfidf = tfidf.transform(X_test)  # i think this computes tf-idf values using the whole test set, but i want each piece to be treated separately
        X_test_tfidf = sp.vstack([vct.transform(np.array([piece])) for piece in X_test])
        
        for clf in classifiers:
            t = datetime.now()
            clf.fit(X_train_tfidf, y_train)
            y_pred = clf.predict(X_test_tfidf)
            clf.runningtime.append((datetime.now()-t).total_seconds())
            clf.cm_sum += confusion_matrix(y_test, y_pred)
            clf.misclassified.append(test_index[np.where(y_test != y_pred)]) # http://stackoverflow.com/a/25570632
            clf.accuracies.append(accuracy_score(y_test, y_pred))
            clf.fones.append(f1_score(y_test, y_pred, average='weighted'))
            clf.fones_micro.append(f1_score(y_test, y_pred, average='micro'))
            clf.fones_macro.append(f1_score(y_test, y_pred, average='macro'))

    result = dict()
    for clf in classifiers:
        clf.misclassified = np.sort(np.hstack(clf.misclassified))
        result[clf.name] = [clf.cm_sum, clf.accuracies, clf.fones, clf.misclassified, filenames[clf.misclassified], clf.runningtime, clf.fones_micro, clf.fones_macro]
    return result

In [533]:
def benchmark_classifiers(composers, NGRAMRANGES, classifiers, vectorizer, n=1, retrieve_title=True):
    misclassified_list = []
    for NGRAMRANGE in NGRAMRANGES:
        print(NGRAMRANGE)
        X, y = build_Xy(composers, size=n)
        cv_result = crossvalidate(X, y, classifiers, vectorizer, NGRAMRANGE)
        for clf, results in cv_result.items():
            print(clf)
            cm = results[0]
            print(cm)
            acc = results[1] # using two different f-measures, don't need this
#             print('accuracy', round(np.mean(acc)*100,2), '({})'.format(round(np.std(acc, ddof=1)*100,2)))
            fones = results[2] # weighted average, don't need this
#             print('f1', round(np.mean(fones)*100,2), '({})'.format(round(np.std(fones, ddof=1)*100,2)), fones)
#             pairwise comparison:
#             print(round(np.mean(fones)*100,2), '({})'.format(round(np.std(fones, ddof=1)*100,2)), '&', round(np.mean(acc)*100,2), '({})'.format(round(np.std(acc, ddof=1)*100,2)))
            misclassified = results[3]
            misclassified_filenames = results[4]
            misclassified_list += list(misclassified_filenames)
#             print('misclassified',[(misclassified[i], misclassified_filenames[i]) for i in range(len(misclassified))])
            runningtime = results[5]
#             print('running time', np.sum(runningtime))
            fones_micro = results[6]
            fones_macro = results[7]
            print(round(np.mean(fones_micro),4), '({})'.format(round(np.std(fones_micro, ddof=1),4)), '&', round(np.mean(fones_macro),4), '({})'.format(round(np.std(fones_macro, ddof=1),4)))
    print('-----')
    return misclassified_list

In [ ]:
# hinge vs squared_hinge: sometimes improves, sometimes worsens
# tol 0.5: sometimes slightly improves, sometimes the opposite
# for now C=5 and loss='hinge' seems to favor BvS, compared to C=1
# Dual doesn't seem to affect result in both svc and lr

# sag solver for lr: definitely worse than liblinear   
# high tolerance favors bach vs scarlatti or beethoven but not handel vs scarlatti. BvS gives higher results so use high tolerance

# elasticnet or l1 doesn't seem to help SGD
# n_iter sometimes does, sometimes harms. test again with default and 10
# loss huber is actually better? # test other losses? 
# ditch SGD. doesn't perform good enough

In [280]:
COMPOSERS = [BACH, BEETHOVEN, DEBUSSY, SCARLATTI, VICTORIA]
NGRAMRANGES = [(1,1),(2,2),(3,3),(4,4),(1,2),(3,4),(1,4)]
CLASSIFIERS = [svm.LinearSVC(penalty='l2', C=5, loss='hinge'),
               linear_model.LogisticRegression(penalty='l2', C=100, tol=1, multi_class='multinomial', solver='sag'),
               neighbors.KNeighborsClassifier(weights='distance'),
               naive_bayes.MultinomialNB(alpha=0.00001, fit_prior=False),
               neural_network.MLPClassifier(solver='lbfgs',hidden_layer_sizes=(10,))]

In [281]:
VECTORIZER = TfidfVectorizer(sublinear_tf=True)
benchmark_classifiers(COMPOSERS,NGRAMRANGES,CLASSIFIERS,VECTORIZER)

(1, 1)
LogisticRegression
[[2164   35    8   31    7]
 [  44  481   37  100    5]
 [  16   17  156    9    1]
 [  30   56   11  498    3]
 [   2    5    4   20  302]]
accuracy 89.09 (1.75) [0.88888888888888884, 0.89629629629629626, 0.90841584158415845, 0.8910891089108911, 0.89356435643564358, 0.86386138613861385, 0.90346534653465349, 0.90099009900990101, 0.90594059405940597, 0.85643564356435642]
f1 89.02 (1.96) [0.88644622269766127, 0.89967191262677781, 0.90801710770960831, 0.88738642077835095, 0.89252910289537246, 0.86100653765044843, 0.90514932005857662, 0.90280650269115215, 0.90734141671213819, 0.85158164716542617]
MLPClassifier
[[2170   37    8   23    7]
 [  30  542   19   74    2]
 [  12   19  157    9    2]
 [  31   65    6  487    9]
 [   1    7    1   12  312]]
accuracy 90.75 (1.56) [0.90123456790123457, 0.92345679012345683, 0.91831683168316836, 0.92079207920792083, 0.90346534653465349, 0.90346534653465349, 0.87128712871287128, 0.89851485148514854, 0.91831683168316836, 0.91584

/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86_64.egg/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


LogisticRegression
[[2129   65    0   44    7]
 [ 297  352    0   17    1]
 [  99   51   24   25    0]
 [ 243   46    0  308    1]
 [  63    1    0    4  265]]
accuracy 76.15 (1.59) [0.75308641975308643, 0.77777777777777779, 0.75247524752475248, 0.78217821782178221, 0.74752475247524752, 0.73762376237623761, 0.75742574257425743, 0.75990099009900991, 0.78712871287128716, 0.75990099009900991]
f1 73.67 (1.87) [0.73848580210235959, 0.75915984178217721, 0.72751556985203192, 0.76050340257241944, 0.72484805004357233, 0.7060998724569747, 0.73123580933513355, 0.72255035101033327, 0.76265748345699658, 0.73372095704484019]
MLPClassifier
[[1686  350   40  122   47]
 [  35  536   39   37   20]
 [   9   96   74    7   13]
 [  32  217   65  251   33]
 [   7   17   11    1  297]]
accuracy 70.36 (6.23) [0.73086419753086418, 0.74567901234567902, 0.78712871287128716, 0.74752475247524752, 0.64356435643564358, 0.6856435643564357, 0.56930693069306926, 0.68316831683168322, 0.73762376237623761, 0.7054455445544

In [282]:
VECTORIZER = CountVectorizer(binary=True)
benchmark_classifiers(COMPOSERS,NGRAMRANGES,CLASSIFIERS,VECTORIZER)

(1, 1)
LogisticRegression
[[2170   16   11   44    4]
 [  64  451   41  106    5]
 [  27   41  104   24    3]
 [  57   90   24  420    7]
 [   6    6    2   20  299]]
accuracy 85.21 (2.46) [0.80740740740740746, 0.84444444444444444, 0.86138613861386137, 0.88613861386138615, 0.82178217821782173, 0.87871287128712872, 0.85148514851485146, 0.83910891089108908, 0.86633663366336633, 0.86386138613861385]
f1 84.91 (2.65) [0.79199547023796091, 0.83681336440860365, 0.85749957683649347, 0.88534748342713698, 0.83470695066590539, 0.87484652783138506, 0.84691248996050428, 0.83557812686829147, 0.86715411057638625, 0.86027181824888743]
MLPClassifier
[[2166   22   15   33    9]
 [  41  509   34   71   12]
 [  21   33  119   19    7]
 [  50   76   18  429   25]
 [   8   13    2   17  293]]
accuracy 86.99 (2.2) [0.85185185185185186, 0.83950617283950613, 0.88613861386138615, 0.88118811881188119, 0.86633663366336633, 0.90099009900990101, 0.86138613861386137, 0.85148514851485146, 0.85643564356435642, 0.90346

/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86_64.egg/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86_64.egg/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86_64.egg/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


LogisticRegression
[[2154   27    7   50    7]
 [ 102  532    1   25    7]
 [  47   52   83   12    5]
 [  68   61    0  467    2]
 [   3    3    0    5  322]]
accuracy 88.03 (2.16) [0.91604938271604941, 0.8691358024691358, 0.85643564356435642, 0.85148514851485146, 0.89603960396039606, 0.90594059405940597, 0.8589108910891089, 0.88366336633663367, 0.88118811881188119, 0.88366336633663367]
f1 87.42 (2.33) [0.91420811655427625, 0.86592231141918685, 0.84407621692973356, 0.84526448798726017, 0.89059640847770472, 0.90025840524759637, 0.85184351316744, 0.88040716332362179, 0.87521014686331733, 0.8745580942318778]
MLPClassifier
[[2207   33    1    2    2]
 [  19  615   10   21    2]
 [  12   11  171    3    2]
 [  27   25    3  541    2]
 [   3    7    1    2  320]]
accuracy 95.35 (0.82) [0.9580246913580247, 0.96296296296296291, 0.9455445544554455, 0.94059405940594054, 0.95544554455445541, 0.96534653465346532, 0.94801980198019797, 0.9455445544554455, 0.95792079207920788, 0.95544554455445541]
f

/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86_64.egg/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86_64.egg/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86_64.egg/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86

LogisticRegression
[[2131   88    0   20    6]
 [ 183  479    0    1    4]
 [  70  123    2    4    0]
 [ 233  143    0  221    1]
 [  29    8    0    0  296]]
accuracy 77.41 (1.66) [0.75555555555555554, 0.76790123456790127, 0.79207920792079212, 0.75, 0.79702970297029707, 0.78465346534653468, 0.78960396039603964, 0.75990099009900991, 0.76485148514851486, 0.77970297029702973]
f1 74.31 (1.79) [0.72503311721832864, 0.73688668890290754, 0.76455240621272602, 0.72417153038835214, 0.76356969752440462, 0.74316534070844675, 0.76714481135114254, 0.72428647030990523, 0.72765515338046116, 0.7540733177704434]
MLPClassifier
[[2134   65   17   24    5]
 [  24  606   17   17    3]
 [   8   32  156    3    0]
 [  18   57    7  510    6]
 [   0    6    1    1  325]]
accuracy 92.31 (1.13) [0.91604938271604941, 0.91604938271604941, 0.90841584158415845, 0.93316831683168322, 0.9356435643564357, 0.93316831683168322, 0.92079207920792083, 0.9356435643564357, 0.90594059405940597, 0.92574257425742579]
f1 92.41 (

...

LogisticRegression
[[2150   81    0   10    4]
 [ 278  385    0    2    2]
 [  93  103    1    2    0]
 [ 324  154    0  120    0]
 [  59    8    0    0  266]]
accuracy 72.29 (2.5) [0.69135802469135799, 0.71851851851851856, 0.72277227722772275, 0.75247524752475248, 0.6856435643564357, 0.69801980198019797, 0.73019801980198018, 0.73514851485148514, 0.75990099009900991, 0.73514851485148514]
f1 67.58 (2.86) [0.63750472061986529, 0.67155871280853641, 0.67595641898142333, 0.70262507332595936, 0.62907627912857944, 0.65315733455101699, 0.68886184696094144, 0.69321094584307519, 0.71904069608205623, 0.68744060104862104]
MLPClassifier
[[2122   73   13   34    3]
 [  23  619   11   13    1]
 [   5   34  157    2    1]
 [  25   75    3  489    6]
 [   1    7    1    3  321]]
accuracy 91.74 (1.35) [0.91604938271604941, 0.93333333333333335, 0.90346534653465349, 0.91089108910891092, 0.92326732673267331, 0.89356435643564358, 0.91089108910891092, 0.92574257425742579, 0.93811881188118806, 0.9183168316831

/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86_64.egg/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86_64.egg/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86_64.egg/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86

LogisticRegression
[[2167   24    0   51    3]
 [ 216  405    0   45    1]
 [ 114   28    1   54    2]
 [ 156   19    0  421    2]
 [  26    1    0    3  303]]
accuracy 81.57 (1.68) [0.81481481481481477, 0.80740740740740746, 0.82673267326732669, 0.80693069306930698, 0.84653465346534651, 0.79950495049504955, 0.8143564356435643, 0.80693069306930698, 0.79455445544554459, 0.83910891089108908]
f1 78.99 (2.04) [0.78917425652452233, 0.77489169016961879, 0.79688921012868297, 0.78581823547537555, 0.82634776487207051, 0.76969775471823954, 0.79346401774183428, 0.77815684412247943, 0.76436502553608499, 0.8203744562871228]
MLPClassifier
[[2202   20   10   10    3]
 [  26  578   12   47    4]
 [   6   13  173    5    2]
 [  20   46    4  519    9]
 [   6    6    1    1  319]]
accuracy 93.79 (3.36) [0.94814814814814818, 0.94320987654320987, 0.95049504950495045, 0.95297029702970293, 0.95792079207920788, 0.94801980198019797, 0.9356435643564357, 0.84653465346534651, 0.96534653465346532, 0.93069306930693

/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86_64.egg/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86_64.egg/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86_64.egg/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86

KeyboardInterrupt: 

In [283]:
VECTORIZER = CountVectorizer()
benchmark_classifiers(COMPOSERS,NGRAMRANGES,CLASSIFIERS,VECTORIZER)

(1, 1)


/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86_64.egg/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86_64.egg/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86_64.egg/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86

LogisticRegression
[[2181   64    0    0    0]
 [ 532  135    0    0    0]
 [ 187   12    0    0    0]
 [ 558   39    0    1    0]
 [ 299   34    0    0    0]]
accuracy 57.32 (3.42) [0.65925925925925921, 0.57283950617283952, 0.5866336633663366, 0.53960396039603964, 0.56930693069306926, 0.54702970297029707, 0.57920792079207917, 0.56188118811881194, 0.54207920792079212, 0.57425742574257421]
f1 44.68 (5.1) [0.57840928956871551, 0.43775580218124593, 0.45732000384504484, 0.40381661928569096, 0.45792079207920794, 0.41256348745173671, 0.45486530031304967, 0.41633342319497313, 0.40522602771453725, 0.44338362407669341]
MLPClassifier
[[2180   30    8   20    7]
 [  39  566   14   41    7]
 [  12   13  163    8    3]
 [  29   49    6  503   11]
 [   8    5    4    5  311]]
accuracy 92.11 (0.77) [0.93086419753086425, 0.9135802469135802, 0.92574257425742579, 0.92821782178217827, 0.92821782178217827, 0.91831683168316836, 0.91831683168316836, 0.92574257425742579, 0.90841584158415845, 0.91336633663366

/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86_64.egg/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86_64.egg/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86_64.egg/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86

LogisticRegression
[[2044  197    0    2    2]
 [ 334  332    0    0    1]
 [ 107   92    0    0    0]
 [ 449  141    0    8    0]
 [ 152  180    0    0    1]]
accuracy 59.01 (2.51) [0.58024691358024694, 0.60246913580246919, 0.57673267326732669, 0.57425742574257421, 0.58168316831683164, 0.60643564356435642, 0.64603960396039606, 0.60396039603960394, 0.56683168316831678, 0.56188118811881194]
f1 49.81 (3.74) [0.48953566596123044, 0.50992388097980934, 0.48313644477802919, 0.47250669817654756, 0.47787604121924981, 0.51614704883662021, 0.59270585073396798, 0.50300285664090583, 0.46769131355637833, 0.46838450231577783]
MLPClassifier
[[2186   32    6   17    4]
 [  32  583    8   40    4]
 [  13   12  165    7    2]
 [  21   48    3  523    3]
 [   3    6    1    4  319]]
accuracy 93.42 (1.71) [0.91851851851851851, 0.92592592592592593, 0.95049504950495045, 0.92079207920792083, 0.94801980198019797, 0.94059405940594054, 0.94801980198019797, 0.95297029702970293, 0.90099009900990101, 0.93564356435

...

LogisticRegression
[[1656  588    0    1    0]
 [  96  570    0    0    1]
 [  31  167    0    1    0]
 [ 201  394    0    3    0]
 [  52  279    0    0    2]]
accuracy 55.2 (5.53) [0.57530864197530862, 0.44444444444444442, 0.60148514851485146, 0.58168316831683164, 0.52970297029702973, 0.55940594059405946, 0.57178217821782173, 0.48267326732673266, 0.54207920792079212, 0.63118811881188119]
f1 50.2 (3.95) [0.49634774762879336, 0.42198665387749207, 0.54605066847997941, 0.50309730543869846, 0.48646396992640445, 0.524983267557525, 0.52396991800020343, 0.4711751596443835, 0.486847874582854, 0.55895523883697507]
MLPClassifier
[[2078  117    8   31   11]
 [  52  569    8   28   10]
 [  17   41  128   13    0]
 [  42   72    4  474    6]
 [   9    9    0    2  313]]
accuracy 88.12 (2.91) [0.87654320987654322, 0.90617283950617289, 0.88118811881188119, 0.84405940594059403, 0.90346534653465349, 0.88861386138613863, 0.86138613861386137, 0.90099009900990101, 0.82920792079207917, 0.92079207920792083]

/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86_64.egg/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86_64.egg/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86_64.egg/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86

LogisticRegression
[[2194   51    0    0    0]
 [ 519  148    0    0    0]
 [ 186   13    0    0    0]
 [ 568   27    0    3    0]
 [ 305   27    0    0    1]]
accuracy 58.04 (2.89) [0.57530864197530862, 0.562962962962963, 0.62128712871287128, 0.54207920792079212, 0.56930693069306926, 0.58168316831683164, 0.57673267326732669, 0.54950495049504955, 0.59158415841584155, 0.63366336633663367]
f1 45.51 (4.45) [0.45256623892639219, 0.44062156429900878, 0.50742028915253834, 0.40259329509045572, 0.43193935710210762, 0.45716288197364729, 0.44770899807223552, 0.40123201281296844, 0.46185119165796268, 0.54764208261298331]
MLPClassifier
[[2178   38    6   17    6]
 [  39  587   13   27    1]
 [   8   14  172    2    3]
 [  30   46    5  512    5]
 [   2    3    2    3  323]]
accuracy 93.32 (1.37) [0.92345679012345683, 0.93580246913580245, 0.9356435643564357, 0.95544554455445541, 0.91584158415841588, 0.92079207920792083, 0.93811881188118806, 0.91831683168316836, 0.9356435643564357, 0.952970297029702

/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86_64.egg/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86_64.egg/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/achim/anaconda/envs/idp/lib/python3.5/site-packages/scikit_learn-0.18.1-py3.5-macosx-10.6-x86_64.egg/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


LogisticRegression
[[1514  703    1   11   16]
 [  83  581    1    1    1]
 [  20  163   16    0    0]
 [  85  474    0   38    1]
 [  19  188    0    3  123]]
accuracy 56.21 (3.45) [0.49135802469135803, 0.5654320987654321, 0.5866336633663366, 0.55693069306930698, 0.5222772277227723, 0.59405940594059403, 0.56683168316831678, 0.59158415841584155, 0.59900990099009899, 0.54702970297029707]
f1 55.93 (3.4) [0.50652242733827713, 0.56122206419837606, 0.56612360648665694, 0.55254480262218908, 0.50296663990860568, 0.57505653879602003, 0.56483819487982057, 0.61765124936145555, 0.58450796316986209, 0.56126727474870075]
MLPClassifier
[[1462  717   12   36   18]
 [  44  586   14   18    5]
 [   9   93   90    7    0]
 [  38  344   14  194    8]
 [   2   41    0    1  289]]
accuracy 64.84 (5.53) [0.65925925925925921, 0.68148148148148147, 0.57178217821782173, 0.6955445544554455, 0.5544554455445545, 0.69059405940594054, 0.61386138613861385, 0.62623762376237624, 0.66831683168316836, 0.72277227722772275

In [405]:
# pairwise classification
NGRAMRANGES = [(1,2)]
VECTORIZER = TfidfVectorizer(sublinear_tf=True)
COMPOSERS = [BACH, BEETHOVEN]
benchmark_classifiers(COMPOSERS,NGRAMRANGES,CLASSIFIERS,VECTORIZER)

(1, 2)
LogisticRegression
97.91 (0.8) & 96.98 (1.15)
MLPClassifier
98.52 (0.63) & 97.86 (0.88)
MultinomialNB
97.12 (1.03) & 95.97 (1.4)
LinearSVC
98.83 (0.65) & 98.3 (0.95)
KNeighborsClassifier
96.39 (0.76) & 94.54 (1.35)
-----


In [406]:
COMPOSERS = [BACH, DEBUSSY]
benchmark_classifiers(COMPOSERS,NGRAMRANGES,CLASSIFIERS,VECTORIZER)

(1, 2)
LogisticRegression
99.14 (0.68) & 96.69 (2.55)
MLPClassifier
99.47 (0.39) & 98.09 (1.44)
MultinomialNB
99.06 (0.43) & 96.34 (2.63)
LinearSVC
99.84 (0.29) & 99.36 (1.12)
KNeighborsClassifier
98.37 (1.23) & 94.16 (3.73)
-----


In [407]:
COMPOSERS = [BACH, SCARLATTI]
benchmark_classifiers(COMPOSERS,NGRAMRANGES,CLASSIFIERS,VECTORIZER)

(1, 2)
LogisticRegression
98.56 (0.73) & 97.8 (1.1)
MLPClassifier
99.26 (0.45) & 98.9 (0.68)
MultinomialNB
97.78 (0.53) & 96.55 (1.12)
LinearSVC
99.61 (0.31) & 99.4 (0.46)
KNeighborsClassifier
97.96 (0.78) & 96.81 (1.2)
-----


In [408]:
COMPOSERS = [BACH, VICTORIA]
benchmark_classifiers(COMPOSERS,NGRAMRANGES,CLASSIFIERS,VECTORIZER)

(1, 2)
LogisticRegression
99.42 (0.49) & 98.69 (1.19)
MLPClassifier
99.53 (0.48) & 98.97 (1.04)
MultinomialNB
99.3 (0.51) & 98.45 (1.15)
LinearSVC
99.81 (0.27) & 99.58 (0.61)
KNeighborsClassifier
99.46 (0.45) & 98.8 (1.03)
-----


In [409]:
COMPOSERS = [BEETHOVEN, DEBUSSY]
benchmark_classifiers(COMPOSERS,NGRAMRANGES,CLASSIFIERS,VECTORIZER)

(1, 2)
LogisticRegression
96.42 (2.21) & 94.96 (2.69)
MLPClassifier
97.57 (2.29) & 96.68 (2.8)
MultinomialNB
87.99 (4.38) & 79.84 (5.52)
LinearSVC
98.73 (1.39) & 98.32 (1.67)
KNeighborsClassifier
91.69 (4.22) & 86.83 (5.41)
-----


In [410]:
COMPOSERS = [BEETHOVEN, SCARLATTI]
benchmark_classifiers(COMPOSERS,NGRAMRANGES,CLASSIFIERS,VECTORIZER)

(1, 2)
LogisticRegression
92.72 (2.41) & 92.63 (2.45)
MLPClassifier
95.73 (2.11) & 95.7 (2.12)
MultinomialNB
80.71 (2.68) & 79.4 (3.28)
LinearSVC
96.28 (2.14) & 96.25 (2.15)
KNeighborsClassifier
94.39 (2.23) & 94.34 (2.29)
-----


In [411]:
COMPOSERS = [BEETHOVEN, VICTORIA]
benchmark_classifiers(COMPOSERS,NGRAMRANGES,CLASSIFIERS,VECTORIZER)

(1, 2)
LogisticRegression
98.2 (1.48) & 97.93 (1.72)
MLPClassifier
99.4 (0.97) & 99.31 (1.15)
MultinomialNB
97.0 (1.76) & 96.59 (2.04)
LinearSVC
99.5 (0.97) & 99.42 (1.16)
KNeighborsClassifier
94.1 (2.18) & 93.46 (2.64)
-----


In [412]:
COMPOSERS = [DEBUSSY, SCARLATTI]
benchmark_classifiers(COMPOSERS,NGRAMRANGES,CLASSIFIERS,VECTORIZER)

(1, 2)
LogisticRegression
98.62 (2.24) & 98.3 (2.46)
MLPClassifier
98.74 (1.19) & 98.32 (1.61)
MultinomialNB
98.75 (1.32) & 98.42 (1.56)
LinearSVC
99.75 (0.53) & 99.68 (0.67)
KNeighborsClassifier
98.37 (1.45) & 97.88 (1.77)
-----


In [413]:
COMPOSERS = [DEBUSSY, VICTORIA]
benchmark_classifiers(COMPOSERS,NGRAMRANGES,CLASSIFIERS,VECTORIZER)

(1, 2)
LogisticRegression
97.74 (2.48) & 97.55 (2.7)
MLPClassifier
98.68 (2.0) & 98.55 (2.19)
MultinomialNB
97.37 (2.22) & 97.17 (2.42)
LinearSVC
99.25 (1.32) & 99.15 (1.46)
KNeighborsClassifier
97.38 (1.99) & 97.14 (2.09)
-----


In [414]:
COMPOSERS = [SCARLATTI, VICTORIA]
benchmark_classifiers(COMPOSERS,NGRAMRANGES,CLASSIFIERS,VECTORIZER)

(1, 2)
LogisticRegression
98.28 (1.45) & 98.14 (1.57)
MLPClassifier
98.93 (1.01) & 98.81 (1.14)
MultinomialNB
98.5 (1.45) & 98.33 (1.64)
LinearSVC
99.36 (1.04) & 99.28 (1.19)
KNeighborsClassifier
94.74 (1.71) & 94.36 (1.83)
-----


In [441]:
# Identify the often-misclassified files
# do the experiment 100 times with the best classifier, SVM
# then find pieces that are misclassified more than 50% of the time
COMPOSERS = [BACH, BEETHOVEN, DEBUSSY, SCARLATTI, VICTORIA]
CLASSIFIERS = [svm.LinearSVC(penalty='l2', C=5, loss='hinge')] # 
appendix = []
for i in range(100):
    appendix += benchmark_classifiers(COMPOSERS,NGRAMRANGES,CLASSIFIERS,VECTORIZER)
Counter(appendix).most_common()

(1, 2)
LinearSVC
[[2230    6    0    6    3]
 [  26  612    4   25    0]
 [   3    2  192    1    1]
 [   9   20    0  568    1]
 [   1    1    0    3  328]]
97.23 (0.59) & 96.45 (0.71)
-----
(1, 2)
LinearSVC
[[2233    5    0    3    4]
 [  23  614    4   26    0]
 [   3    2  192    1    1]
 [   9   21    1  567    0]
 [   1    1    0    4  327]]
97.3 (0.74) & 96.41 (1.17)
-----
(1, 2)
LinearSVC
[[2234    5    0    4    2]
 [  24  616    3   24    0]
 [   4    6  187    1    1]
 [   9   20    0  568    1]
 [   0    1    0    5  327]]
97.28 (0.96) & 96.37 (1.53)
-----
(1, 2)
LinearSVC
[[2235    4    0    3    3]
 [  20  618    4   25    0]
 [   4    4  189    1    1]
 [   9   19    1  569    0]
 [   0    1    0    3  329]]
97.48 (0.72) & 96.65 (1.21)
-----
(1, 2)
LinearSVC
[[2231    7    0    3    4]
 [  23  620    4   20    0]
 [   3    3  191    1    1]
 [   6   16    0  575    1]
 [   3    1    0    3  326]]
97.55 (0.36) & 96.78 (0.63)
-----
(1, 2)
LinearSVC
[[2231    6    0    4   

[('victoria-da_falsobordon-4_(c)alvarez.mxl', 100),
 ('beethoven_biamonti_270_(c)orlandi.mxl', 100),
 ('beethoven_symphony_6_68_4_(nc)wittenburg.mxl', 100),
 ('beethoven_biamonti_459_(c)orlandi.mxl', 100),
 ('scarlatti_sonata_k-166_(c)sankey.mxl', 100),
 ('beethoven_biamonti_491_(c)orlandi.mxl', 100),
 ('beethoven_symphony_10_b_838_2_idea_by_(c)prengel.mxl', 100),
 ('beethoven_cello_sonata_5_2_(c)harfesoft.mxl', 100),
 ('bach-js_canons_bwv-1074_(grossman).xml', 100),
 ('bach-js_wtk_bwv-876frag_(c)tomita.xml', 100),
 ('beethoven_string_quartet_133_(c)harfesoft.mxl', 100),
 ('beethoven_biamonti_461h_(c)orlandi.mxl', 100),
 ('scarlatti-d_esserciso_3_(c)icking-archive.mxl', 100),
 ('debussy_epigraphes_antiques_1_(c)lefeldt.mxl', 100),
 ('victoria-de_o_magnum_mysterium_(brass)_(c)icking-archive.mxl', 100),
 ('scarlatti_sonata_k-308_(c)sankey.mxl', 100),
 ('beethoven_sketch_1_(c)orlandi.mxl', 100),
 ('beethoven_piano_sonata_31-3_4_(nc)wittenburg.mxl', 100),
 ('scarlatti_sonata_k-340_(c)sanke

In [546]:
# Use both chord sequences and duration data
flatten = lambda l: [item for sublist in l for item in sublist]
with open('beethoven-durations.txt', 'r') as f:
    BD = [line.strip() for line in f.readlines()]
with open('scarlatti-durations.txt', 'r') as f:
    SD = [line.strip() for line in f.readlines()]
with open('bach-durations.txt', 'r') as f:
    BD2 = [line.strip() for line in f.readlines()]
with open('debussy-durations.txt', 'r') as f:
    DD = [line.strip() for line in f.readlines()]
with open('victoria-durations.txt', 'r') as f:
    VD = [line.strip() for line in f.readlines()]
    
BD_TYPELENGTH = [piece.split(';') for piece in BD]
SD_TYPELENGTH = [piece.split(';') for piece in SD]
BD2_TYPELENGTH = [piece.split(';') for piece in BD2]
DD_TYPELENGTH = [piece.split(';') for piece in DD]
VD_TYPELENGTH = [piece.split(';') for piece in VD]

typelengths = list(set(flatten(BD2_TYPELENGTH+BD_TYPELENGTH+DD_TYPELENGTH+SD_TYPELENGTH+VD_TYPELENGTH)))
typelength_dict = {typelengths[i]: str(i+300) for i in range(len(typelengths))}
BD_T = [(' '.join([typelength_dict[dur] for dur in piece]),'temp') for piece in BD_TYPELENGTH]
SD_T = [(' '.join([typelength_dict[dur] for dur in piece]),'temp') for piece in SD_TYPELENGTH]
BD2_T = [(' '.join([typelength_dict[dur] for dur in piece]),'temp') for piece in BD2_TYPELENGTH]
DD_T = [(' '.join([typelength_dict[dur] for dur in piece]),'temp') for piece in DD_TYPELENGTH]
VD_T = [(' '.join([typelength_dict[dur] for dur in piece]),'temp') for piece in VD_TYPELENGTH]

In [600]:
BD_LENGTHONLY = [[string.split('|')[1] for string in piece.split(';')] for piece in BD]
SD_LENGTHONLY = [[string.split('|')[1] for string in piece.split(';')] for piece in SD]
BD2_LENGTHONLY = [[string.split('|')[1] for string in piece.split(';')] for piece in BD2]
DD_LENGTHONLY = [[string.split('|')[1] for string in piece.split(';')] for piece in DD]
VD_LENGTHONLY = [[string.split('|')[1] for string in piece.split(';')] for piece in VD]
lengths = list(set(flatten(BD2_LENGTHONLY+BD_LENGTHONLY+DD_LENGTHONLY+SD_LENGTHONLY+VD_LENGTHONLY)))
length_dict = {lengths[i]: str(i+200) for i in range(len(lengths))}
len(lengths)
# BD_L = [(' '.join([length_dict[dur] for dur in piece]),'temp') for piece in BD_LENGTHONLY]
# BD2_L = [(' '.join([length_dict[dur] for dur in piece]),'temp') for piece in BD2_LENGTHONLY]
# SD_L = [(' '.join([length_dict[dur] for dur in piece]),'temp') for piece in SD_LENGTHONLY]
# DD_L = [(' '.join([length_dict[dur] for dur in piece]),'temp') for piece in DD_LENGTHONLY]
# VD_L = [(' '.join([length_dict[dur] for dur in piece]),'temp') for piece in VD_LENGTHONLY]
duration_all       = flatten(BD2_LENGTHONLY+BD_LENGTHONLY+DD_LENGTHONLY+SD_LENGTHONLY+VD_LENGTHONLY)
duration_bach      = flatten(BD2_LENGTHONLY)
duration_beethoven = flatten(BD_LENGTHONLY)
duration_debussy   = flatten(DD_LENGTHONLY)
duration_scarlatti = flatten(SD_LENGTHONLY)
duration_victoria  = flatten(VD_LENGTHONLY)

In [606]:
for l in [duration_all,duration_bach,duration_beethoven,duration_debussy,duration_scarlatti,duration_victoria]:
    for key, value in Counter(l).most_common(10):
        print(key, '&', round(100*value/len(l),2))
    print('')

Eighth & 25.24
Quarter & 22.27
16th & 18.34
Dotted Half & 8.8
Half & 5.72
Whole & 4.38
32nd & 3.42
Dotted Quarter & 2.35
Eighth Triplet (1/3 QL) & 2.2
16th Triplet (1/6 QL) & 1.7

Eighth & 28.19
16th & 22.4
Quarter & 19.99
Dotted Half & 8.81
Half & 4.41
32nd & 3.14
Whole & 3.01
Dotted Quarter & 2.01
Eighth Triplet (1/3 QL) & 1.67
16th Triplet (1/6 QL) & 1.36

Eighth & 25.5
Quarter & 19.67
16th & 19.51
Half & 6.78
Whole & 5.05
Dotted Half & 4.61
32nd & 4.03
Dotted Quarter & 3.36
Eighth Triplet (1/3 QL) & 2.82
16th Triplet (1/6 QL) & 2.14

Eighth & 20.22
16th & 18.07
Quarter & 14.25
32nd & 9.96
16th Triplet (1/6 QL) & 5.63
Half & 4.44
Eighth Triplet (1/3 QL) & 4.01
Dotted Half & 3.32
Dotted Quarter & 2.86
Whole & 2.81

Quarter & 33.61
Dotted Half & 22.42
Eighth & 19.03
16th & 6.68
Whole & 5.88
Half & 4.21
Eighth Triplet (1/3 QL) & 2.19
Dotted Eighth & 1.46
32nd & 1.3
16th Triplet (1/6 QL) & 0.76

Quarter & 44.92
Half & 21.8
Eighth & 16.03
Whole & 11.82
Dotted Quarter & 2.91
Dotted Half &

In [569]:
def crossvalidate_twofeaturesets(X_tuple1, X_tuple2, y, classifiers, vectorizer, range1, range2, K=10):    
    for clf in classifiers:
        clf.cm_sum = np.zeros([len(set(y)),len(set(y))], dtype='int16')
        clf.accuracies, clf.fones, clf.misclassified, clf.runningtime = [], [], [], []
        clf.fones_micro, clf.fones_macro = [], []
        clf.name = str(clf).split('(')[0]

    X1 = np.array([piece[0] for piece in X_tuple1])
    X2 = np.array([piece[0] for piece in X_tuple2])
    filenames = np.array([piece[1] for piece in X_tuple2])
    kf = KFold(n_splits=K, shuffle=True)
    for train_index, test_index in kf.split(y):
        y_train, y_test = y[train_index], y[test_index]
        X_train_new, X_test_new = X1[train_index], X1[test_index]
        vct1 = vectorizer.set_params(ngram_range=range1)
        X_train, X_test = X2[train_index], X2[test_index] 
        vct2 = vectorizer.set_params(ngram_range=range2)
   
        X_train_new_tfidf = vct1.fit_transform(X_train_new) # use two separate vectorizers for each feature set
        X_test_new_tfidf = sp.vstack([vct1.transform(np.array([piece])) for piece in X_test_new])
        X_train_tfidf = vct2.fit_transform(X_train)
        X_test_tfidf = sp.vstack([vct2.transform(np.array([piece])) for piece in X_test])
        
        X_train_tfidf = sp.hstack((X_train_tfidf, X_train_new_tfidf)) # Merge the two feature sets
        X_test_tfidf = sp.hstack((X_test_tfidf, X_test_new_tfidf))
        
        for clf in classifiers:
            t = datetime.now()
            clf.fit(X_train_tfidf, y_train)
            y_pred = clf.predict(X_test_tfidf)
            clf.runningtime.append((datetime.now()-t).total_seconds())
            clf.cm_sum += confusion_matrix(y_test, y_pred)
            clf.misclassified.append(test_index[np.where(y_test != y_pred)]) # http://stackoverflow.com/a/25570632
            clf.accuracies.append(accuracy_score(y_test, y_pred))
            clf.fones.append(f1_score(y_test, y_pred, average='weighted'))
            clf.fones_micro.append(f1_score(y_test, y_pred, average='micro'))
            clf.fones_macro.append(f1_score(y_test, y_pred, average='macro'))

    result = dict()
    for clf in classifiers:
        clf.misclassified = np.sort(np.hstack(clf.misclassified))
        result[clf.name] = [clf.cm_sum, clf.accuracies, clf.fones, clf.misclassified, filenames[clf.misclassified], clf.runningtime, clf.fones_micro, clf.fones_macro]
    return result

In [575]:
def benchmark_classifiers_twofeaturesets(composers1, composers2, range1, range2, classifiers, vectorizer, n=1, retrieve_title=True):
    misclassified_list = []
    print('duration n-gram range:', range1, 'chord n-gram range:', range2)
    X1, y = build_Xy(composers1, size=n)
    X2, y = build_Xy(composers2, size=n)
    cv_result = crossvalidate_twofeaturesets(X1, X2, y, classifiers, vectorizer, range1, range2)
    for clf, results in cv_result.items():
        print(clf)
        cm = results[0]
        print(cm)
        acc = results[1]
        fones = results[2]
        misclassified = results[3]
        misclassified_filenames = results[4]
        misclassified_list += list(misclassified_filenames)
#             print('misclassified',[(misclassified[i], misclassified_filenames[i]) for i in range(len(misclassified))])
        runningtime = results[5]
#         print('running time', np.sum(runningtime))
        fones_micro = results[6]
        fones_macro = results[7]
        print('F-measures')
        print(round(np.mean(fones_micro),4), '({})'.format(round(np.std(fones_micro, ddof=1),4)), '&', round(np.mean(fones_macro),4), '({})'.format(round(np.std(fones_macro, ddof=1),4)))
    print('-----')
    return misclassified_list

In [572]:
CLASSIFIERS = [svm.LinearSVC(penalty='l2', C=5, loss='hinge'),
               linear_model.LogisticRegression(penalty='l2', C=100, tol=1, multi_class='multinomial', solver='sag'),
               neighbors.KNeighborsClassifier(weights='distance'),
               naive_bayes.MultinomialNB(alpha=0.00001, fit_prior=False),
               neural_network.MLPClassifier(solver='lbfgs',hidden_layer_sizes=(10,))]
VECTORIZER = TfidfVectorizer(sublinear_tf=True, lowercase=False, token_pattern=u"(?u)\\b\\w+\\b")

COMPOSERS1 = [BD2_T,BD_T,DD_T,SD_T,VD_T]
COMPOSERS2 = [BACH, BEETHOVEN, DEBUSSY, SCARLATTI, VICTORIA]
benchmark_classifiers_twofeaturesets(COMPOSERS1, COMPOSERS2, (1,1), (1,2), CLASSIFIERS, VECTORIZER)

duration n-gram range: (1, 1) chord n-gram range: (1, 2)
LogisticRegression
[[2176   33    3   25    8]
 [  38  592   17   15    5]
 [   2   18  176    3    0]
 [  18   23    1  556    0]
 [   3    5    0    0  325]]
running time 0.962638
F-measures
0.9463 (0.0206) & 0.9279 (0.0301)
MLPClassifier
[[2218    8    1   14    4]
 [  20  631    6   10    0]
 [   2   11  183    3    0]
 [   4   17    2  575    0]
 [   1    1    0    0  331]]
running time 48.590293
F-measures
0.9743 (0.0071) & 0.963 (0.0091)
MultinomialNB
[[2168   42    4   13   18]
 [  22  628    7    4    6]
 [   1   65  133    0    0]
 [  17  104    2  475    0]
 [   3    8    0    0  322]]
running time 0.448548
F-measures
0.9218 (0.0174) & 0.8783 (0.0324)
LinearSVC
[[2236    5    1    2    1]
 [  25  639    1    2    0]
 [   1    3  195    0    0]
 [   5   11    0  582    0]
 [   0    0    0    0  333]]
running time 18.166799
F-measures
0.9859 (0.0055) & 0.9841 (0.006)
KNeighborsClassifier
[[2178   17    0   36   14]
 [ 13

['bach_art_of_fugue_12_[live]_(c)simonetto.xml',
 'bach_cantata_199_bwv-199_1_(c)icking-archive.xml',
 'bach_cantata_4_bwv-4_8_(c)icking-archive.xml',
 'bach_cantata_5_bwv-5_2_(c)icking-archive.xml',
 'bach_cantata_5_bwv-5_6_(c)icking-archive.xml',
 'bach_cantate_140_bwv-140_7_(c)icking-archive.xml',
 'bach_cantate_172_bwv-172_3_(c)icking-archive.xml',
 'bach_cantate_199_bwv-199_1_(c)icking-archive.xml',
 'bach_cantate_199_bwv-199_3_(c)icking-archive.xml',
 'bach_cantate_199_bwv-199_5_(c)icking-archive.xml',
 'bach-js_491a_ave_maria_(nc)smythe.xml',
 'bach-js_anna-magdalena_10_(nc)werths.xml',
 'bach-js_anna-magdalena_15_(nc)werths.xml',
 'bach-js_anna-magdalena_19_(nc)werths.xml',
 'bach-js_anna-magdalena_31_(nc)werths.xml',
 'bach-js_canons_bwv-1074_(grossman).xml',
 'bach-js_canons_bwv-1074i_(grossman).xml',
 'bach-js_cantata_199_2_(c)icking-archive.xml',
 'bach-js_cantata_199_3_(c)icking-archive.xml',
 'bach-js_cantata_bwv-035_7_(c)langbein.xml',
 'bach-js_chorales_004311b_(c)green

In [576]:
COMPOSERS1 = [BD2_T,BD_T]
COMPOSERS2 = [BACH, BEETHOVEN]
benchmark_classifiers_twofeaturesets(COMPOSERS1, COMPOSERS2, (1,1), (1,2), CLASSIFIERS, VECTORIZER)

duration n-gram range: (1, 1) chord n-gram range: (1, 2)
LogisticRegression
[[2195   50]
 [  44  623]]
F-measures
0.9677 (0.0075) & 0.9542 (0.0111)
MLPClassifier
[[2231   14]
 [  24  643]]
F-measures
0.9869 (0.006) & 0.9814 (0.0086)
MultinomialNB
[[2193   52]
 [  31  636]]
F-measures
0.9715 (0.013) & 0.9604 (0.0174)
LinearSVC
[[2237    8]
 [  21  646]]
F-measures
0.99 (0.0055) & 0.9859 (0.0075)
KNeighborsClassifier
[[2231   14]
 [ 156  511]]
F-measures
0.9416 (0.0159) & 0.9105 (0.0231)
-----


['bach_art_of_fugue_12_[live]_(c)simonetto.xml',
 'bach_art_of_fugue_contrapunctum_13_(nc)wittenburg.xml',
 'bach_brandeburg_concert_1048_2-3_(nc)anonym.xml',
 'bach_cantata_5_bwv-5_2_(c)icking-archive.xml',
 'bach_cantate_172_bwv-172_3_(c)icking-archive.xml',
 'bach_cantate_34_bwv-34_4_(c)icking-archive.xml',
 'bach_concerto_for_four-harpischords_1065_2_(nc)wittenburg.xml',
 'bach_inventions_772-786_[complete]_kirnberger-iii.xml',
 'bach-js_491a_ave_maria_(nc)smythe.xml',
 'bach-js_anna-magdalena_13_(nc)werths.xml',
 'bach-js_anna-magdalena_19_(nc)werths.xml',
 'bach-js_anna-magdalena_9_(nc)werths.xml',
 'bach-js_cantata_199_5_(c)icking-archive.xml',
 'bach-js_capricci_bwv-992_(c)sankey.xml',
 'bach-js_chorales_007903bs_(c)greentree.xml',
 'bach-js_chorales_017206vn_(c)greentree.xml',
 'bach-js_goldberg_variations_988_10_(c)degiusti.xml',
 'bach-js_kunst_der_fuge_12_(c)simonetto.xml',
 'bach-js_kunst_der_fuge_15_(c)unknown[2].xml',
 'bach-js_kunst_der_fuge_15_(unknown).xml',
 'bach-js

In [577]:
COMPOSERS1 = [BD2_T,DD_T]
COMPOSERS2 = [BACH, DEBUSSY]
benchmark_classifiers_twofeaturesets(COMPOSERS1, COMPOSERS2, (1,1), (1,2), CLASSIFIERS, VECTORIZER)

duration n-gram range: (1, 1) chord n-gram range: (1, 2)
LogisticRegression
[[2236    9]
 [  15  184]]
F-measures
0.9902 (0.0059) & 0.9652 (0.0196)
MLPClassifier
[[2241    4]
 [   8  191]]
F-measures
0.9951 (0.0042) & 0.9817 (0.017)
MultinomialNB
[[2232   13]
 [   8  191]]
F-measures
0.9914 (0.0049) & 0.9684 (0.0214)
LinearSVC
[[2242    3]
 [   5  194]]
F-measures
0.9967 (0.0032) & 0.9879 (0.0129)
KNeighborsClassifier
[[2245    0]
 [  67  132]]
F-measures
0.9726 (0.0097) & 0.8847 (0.0491)
-----


['bach_cello_suites_1012_(c)harfesoft.xml',
 'bach-js_491a_ave_maria_(nc)smythe.xml',
 'bach-js_english_suites_bwv-808_4_(c)siu.xml',
 'bach-js_organ_achtzehn_choraele_bwv-662_(c)mccoy.xml',
 'bach-js_organ_klavier_ubung_bwv-681_(c)luquet.xml',
 'bach-js_orgel_buechlein_bwv-613_(c)mccoy.xml',
 'bach-js_suite_for_unaccompanied_cello_bwv-1012_4_(c)bachovich.xml',
 'bach-js_well-tempered-clavier-i_bwv-848_(c)sankey.xml',
 'bach-js_well-tempered-clavier-i_bwv-853a_(c)bachovich.xml',
 'debussy_arabesque_1_(c)dery.mxl',
 'debussy_childrens_corner_1_(c)dery.mxl',
 'debussy_childrens_corner_2_(c)oguri.mxl',
 'debussy_childrens_corner_5_(c)oguri.mxl',
 'debussy_epigraphes_antiques_1_(c)lefeldt.mxl',
 'debussy_epigraphes_antiques_6_(c)lefeldt.mxl',
 'debussy_la_plus_que_lente_(c)lefeldt.mxl',
 'debussy_petite_suite_3_(c)gray.mxl',
 'debussy_petite_suite_3_(c)oguri.mxl',
 'debussy_petite_suite_3_(c)yogore.mxl',
 'debussy_petite_suite_4_(c)yogore.mxl',
 'debussy_petit_negre_(c)yogore.mxl',
 'debus

In [578]:
COMPOSERS1 = [BD2_T,SD_T]
COMPOSERS2 = [BACH, SCARLATTI]
benchmark_classifiers_twofeaturesets(COMPOSERS1, COMPOSERS2, (1,1), (1,2), CLASSIFIERS, VECTORIZER)

duration n-gram range: (1, 1) chord n-gram range: (1, 2)
LogisticRegression
[[2216   29]
 [  48  550]]
F-measures
0.9729 (0.018) & 0.9586 (0.0283)
MLPClassifier
[[2231   14]
 [  16  582]]
F-measures
0.9894 (0.0076) & 0.9844 (0.0108)
MultinomialNB
[[2219   26]
 [  40  558]]
F-measures
0.9768 (0.0101) & 0.9645 (0.0158)
LinearSVC
[[2241    4]
 [   7  591]]
F-measures
0.9961 (0.0045) & 0.9943 (0.0066)
KNeighborsClassifier
[[2217   28]
 [  51  547]]
F-measures
0.9722 (0.0112) & 0.9573 (0.0167)
-----


['bach_cantata_199_bwv-199_1_(c)icking-archive.xml',
 'bach_cantata_199_bwv-199_2_(c)icking-archive.xml',
 'bach_cantata_199_bwv-199_5_(c)icking-archive.xml',
 'bach_cantata_5_bwv-5_6_(c)icking-archive.xml',
 'bach_cantate_199_bwv-199_1_(c)icking-archive.xml',
 'bach_cantate_199_bwv-199_5_(c)icking-archive.xml',
 'bach-js_cantata_199_5_(c)icking-archive.xml',
 'bach-js_cantata_bwv-006_1_(c)langbein.xml',
 'bach-js_concertos_bwv-973_(c)sankey.xml',
 'bach-js_inventions_bwv-773_(c)sankey.xml',
 'bach-js_inventions_bwv-774_(c)sankey.xml',
 'bach-js_inventions_bwv-776_(c)sankey.xml',
 'bach-js_inventions_bwv-777_(c)sankey.xml',
 'bach-js_inventions_bwv-780_(c)sankey.xml',
 'bach-js_inventions_bwv-781_(c)sankey.xml',
 'bach-js_inventions_bwv-785_(c)sankey.xml',
 'bach-js_inventions_bwv-786_(c)sankey.xml',
 'bach-js_orgelbuechlein_28_(c)icking-archive.xml',
 'bach-js_partie_bwv-832_(c)sankey.xml',
 'bach-js_sinfonias_bwv-792_(c)sankey.xml',
 'bach-js_sinfonias_bwv-800_(c)sankey.xml',
 'bach-

In [579]:
COMPOSERS1 = [BD2_T,VD_T]
COMPOSERS2 = [BACH, VICTORIA]
benchmark_classifiers_twofeaturesets(COMPOSERS1, COMPOSERS2, (1,1), (1,2), CLASSIFIERS, VECTORIZER)

duration n-gram range: (1, 1) chord n-gram range: (1, 2)
LogisticRegression
[[2237    8]
 [   6  327]]
F-measures
0.9946 (0.0052) & 0.9883 (0.0109)
MLPClassifier
[[2244    1]
 [   4  329]]
F-measures
0.9981 (0.0038) & 0.9959 (0.0081)
MultinomialNB
[[2228   17]
 [   9  324]]
F-measures
0.9899 (0.0045) & 0.9779 (0.0099)
LinearSVC
[[2245    0]
 [   0  333]]
F-measures
1.0 (0.0) & 1.0 (0.0)
KNeighborsClassifier
[[2234   11]
 [   3  330]]
F-measures
0.9946 (0.0049) & 0.9878 (0.0111)
-----


['bach-js_canons_bwv-1074_(grossman).xml',
 'bach-js_canons_bwv-1074i_(grossman).xml',
 'bach-js_chorales_001007b_(c)greentree.xml',
 'bach-js_chorales_004311b_(c)greentree.xml',
 'bach-js_organ_other_works_bwv-589_(c)mccormick.xml',
 'bach-js_suite_for_unaccompanied_cello_bwv-1010_1_(c)grossman.xml',
 'bach-js_suite_for_unaccompanied_cello_bwv-1010_2_(c)grossman.xml',
 'bach-js_suite_for_unaccompanied_cello_bwv-1012_4_(c)grossman.xml',
 'victoria-da_falsobordon-3_(c)alvarez.mxl',
 'victoria-da_laudate_pueri_bis_(c)alvarez.mxl',
 'victoria-da_laud-et_iesum_(c)alvarez.mxl',
 'victoria-da_o_magnum_mysterium-5-benedictus_(c)alvarez.mxl',
 'victoria-da_victimae_paschale_(c)alvarez.mxl',
 'victoria-de_ave_maria_(brass)_(c)icking-archive.mxl',
 'bach-js_canons_bwv-1074_(grossman).xml',
 'victoria-da_falsobordon-4_(c)alvarez.mxl',
 'victoria-da_laud-et_iesum_(c)alvarez.mxl',
 'victoria-da_victimae_paschale_(c)alvarez.mxl',
 'victoria-de_ave_maria_(brass)_(c)icking-archive.mxl',
 'bach-js_cano

In [580]:
COMPOSERS1 = [BD_T,DD_T]
COMPOSERS2 = [BEETHOVEN, DEBUSSY]
benchmark_classifiers_twofeaturesets(COMPOSERS1, COMPOSERS2, (1,1), (1,2), CLASSIFIERS, VECTORIZER)

duration n-gram range: (1, 1) chord n-gram range: (1, 2)
LogisticRegression
[[646  21]
 [ 21 178]]
F-measures
0.9515 (0.0143) & 0.9312 (0.0172)
MLPClassifier
[[663   4]
 [ 13 186]]
F-measures
0.9804 (0.0173) & 0.9728 (0.023)
MultinomialNB
[[657  10]
 [ 68 131]]
F-measures
0.9099 (0.0367) & 0.8576 (0.0534)
LinearSVC
[[665   2]
 [  5 194]]
F-measures
0.9919 (0.0122) & 0.9891 (0.016)
KNeighborsClassifier
[[665   2]
 [ 93 106]]
F-measures
0.8904 (0.0473) & 0.81 (0.0857)
-----


['beethoven_6_bagattelles_126_(c)hisamori.mxl',
 'beethoven_biamonti_458_(c)orlandi.mxl',
 'beethoven_biamonti_698_(c)orlandi.mxl',
 'beethoven_biamonti_770_(c)orlandi.mxl',
 'beethoven_duo_woo-27-1_(c)orlandi.mxl',
 'beethoven_minor_works_hess-234_(c)unheard-beethoven.mxl',
 'beethoven_piano_concerto_15_2_(c)nemitz.mxl',
 'beethoven_piano_sonata_14-2_1_(nc)wittenburg.mxl',
 'beethoven_piano_sonata_31-3_2_(nc)wittenburg.mxl',
 'beethoven_quartet_8_2_(c)edwards.mxl',
 'beethoven_quartet_8_3_(c)edwards.mxl',
 'beethoven_quartet_9_1_(c)edwards.mxl',
 'beethoven_symphony_10_b_838_1ex_(c)prengel.mxl',
 'beethoven_symphony_10_b_838_2_idea_by_(c)prengel.mxl',
 'beethoven_symphony_6_68_4_(nc)wittenburg.mxl',
 'beethoven_symphony_7_2_(c)cvikl.mxl',
 'beethoven_symphony_7_3_(c)tadokoro.mxl',
 'beethoven_symphony_9_1_(c)cvikl.mxl',
 'beethoven_symphony_9_4_(c)cvikl.mxl',
 'beethoven_variations_cor_pio_non_mi_sento_(c)oguri.mxl',
 'beethoven_waltz_anhang_3_(c)blondel.mxl',
 'debussy_arabesque_2_(c

In [581]:
COMPOSERS1 = [BD_T,SD_T]
COMPOSERS2 = [BEETHOVEN, SCARLATTI]
benchmark_classifiers_twofeaturesets(COMPOSERS1, COMPOSERS2, (1,1), (1,2), CLASSIFIERS, VECTORIZER)

duration n-gram range: (1, 1) chord n-gram range: (1, 2)
LogisticRegression
[[655  12]
 [ 39 559]]
F-measures
0.9597 (0.0126) & 0.9589 (0.0132)
MLPClassifier
[[655  12]
 [ 11 587]]
F-measures
0.9818 (0.014) & 0.9817 (0.014)
MultinomialNB
[[661   6]
 [117 481]]
F-measures
0.9027 (0.0185) & 0.9001 (0.0186)
LinearSVC
[[660   7]
 [ 16 582]]
F-measures
0.9818 (0.0163) & 0.9817 (0.0164)
KNeighborsClassifier
[[649  18]
 [ 24 574]]
F-measures
0.9668 (0.0174) & 0.9665 (0.0175)
-----


['beethoven_6610d_rondo_a_capriccio_129_(nc)smythe.mxl',
 'beethoven_biamonti_360_(c)orlandi.mxl',
 'beethoven_biamonti_456_2_(c)orlandi.mxl',
 'beethoven_biamonti_66_(c)orlandi.mxl',
 'beethoven-laviano_concerto_0_woo-4_(c)laviano.mxl',
 'beethoven_minor_works_hess-250_(c)unheard-beethoven.mxl',
 'beethoven_minor_works_woo-186_(c)unheard-beethoven.mxl',
 'beethoven_quartet_6_4_(c)edwards.mxl',
 'beethoven_quartet_8_3_(c)edwards.mxl',
 'beethoven_song_hess-13_(c)orlandi.mxl',
 'beethoven_symphony_6_1_(c)lucarelli.mxl',
 'beethoven_waltz_woo_84_(c)orlandi.mxl',
 'scarlatti-a_fugue_e_(c)simonetto.mxl',
 'scarlatti-d_esserciso_14_(c)icking-archive.mxl',
 'scarlatti-d_esserciso_15_(c)icking-archive.mxl',
 'scarlatti-d_esserciso_16_(c)icking-archive.mxl',
 'scarlatti-d_esserciso_1_(c)icking-archive.mxl',
 'scarlatti-d_esserciso_21_(c)icking-archive.mxl',
 'scarlatti-d_esserciso_23_(c)icking-archive.mxl',
 'scarlatti-d_esserciso_26_(c)icking-archive.mxl',
 'scarlatti-d_esserciso_28_(c)icking

In [582]:
COMPOSERS1 = [BD_T,VD_T]
COMPOSERS2 = [BEETHOVEN, VICTORIA]
benchmark_classifiers_twofeaturesets(COMPOSERS1, COMPOSERS2, (1,1), (1,2), CLASSIFIERS, VECTORIZER)

duration n-gram range: (1, 1) chord n-gram range: (1, 2)
LogisticRegression
[[657  10]
 [  3 330]]
F-measures
0.987 (0.0149) & 0.9854 (0.0167)
MLPClassifier
[[666   1]
 [  2 331]]
F-measures
0.997 (0.0067) & 0.9964 (0.008)
MultinomialNB
[[658   9]
 [ 11 322]]
F-measures
0.98 (0.0133) & 0.977 (0.0158)
LinearSVC
[[667   0]
 [  1 332]]
F-measures
0.999 (0.0032) & 0.9988 (0.0037)
KNeighborsClassifier
[[644  23]
 [  0 333]]
F-measures
0.977 (0.0116) & 0.9742 (0.0135)
-----


['beethoven_bagattelle_119_2_hess-a21_(c)orlandi.mxl',
 'beethoven_biamonti_488_(c)orlandi.mxl',
 'beethoven_contra-dance_woo-14_11_(c)tadokoro.mxl',
 'beethoven_minor_works_hess-237-1_(c)unheard-beethoven.mxl',
 'beethoven_minor_works_hess-237-3_(c)unheard-beethoven.mxl',
 'beethoven_minor_works_hess-239-2_(c)unheard-beethoven.mxl',
 'beethoven_minor_works_hess-254_(c)unheard-beethoven.mxl',
 'beethoven_minor_works_hess-275_(c)unheard-beethoven.mxl',
 'beethoven_minor_works_woo-62-2_(c)prengel.mxl',
 'beethoven_quartet_14_2_(c)edwards.mxl',
 'victoria-da_o_magnum_mysterium-5-benedictus_(c)alvarez.mxl',
 'victoria-da_victimae_paschale_(c)alvarez.mxl',
 'victoria-de_ave_maria_(brass)_(c)icking-archive.mxl',
 'beethoven_minor_works_hess-275_(c)unheard-beethoven.mxl',
 'victoria-da_laud-et_iesum_(c)alvarez.mxl',
 'victoria-da_victimae_paschale_(c)alvarez.mxl',
 'beethoven_biamonti_466_(c)orlandi.mxl',
 'beethoven_biamonti_473_(c)orlandi.mxl',
 'beethoven_biamonti_488_(c)orlandi.mxl',
 'be

In [583]:
COMPOSERS1 = [DD_T,SD_T]
COMPOSERS2 = [DEBUSSY, SCARLATTI]
benchmark_classifiers_twofeaturesets(COMPOSERS1, COMPOSERS2, (1,1), (1,2), CLASSIFIERS, VECTORIZER)

duration n-gram range: (1, 1) chord n-gram range: (1, 2)
LogisticRegression
[[183  16]
 [  4 594]]
F-measures
0.9749 (0.0289) & 0.9628 (0.0504)
MLPClassifier
[[194   5]
 [  2 596]]
F-measures
0.9912 (0.0085) & 0.9881 (0.0116)
MultinomialNB
[[197   2]
 [ 18 580]]
F-measures
0.975 (0.0167) & 0.9669 (0.0196)
LinearSVC
[[195   4]
 [  1 597]]
F-measures
0.9937 (0.0066) & 0.9916 (0.0088)
KNeighborsClassifier
[[186  13]
 [  1 597]]
F-measures
0.9825 (0.0106) & 0.9731 (0.0199)
-----


['debussy_arabesque_1_(c)dery.mxl',
 'debussy_childrens_corner_1_(c)dery.mxl',
 'debussy_childrens_corner_4_(c)dery.mxl',
 'debussy_childrens_corner_5_(c)dery.mxl',
 'debussy_epigraphes_antiques_1_(c)lefeldt.mxl',
 'debussy_images_2_2_(c)dery.mxl',
 'debussy_la_plus_que_lente_(c)lefeldt.mxl',
 'debussy_petit_negre_(c)yogore.mxl',
 'debussy_pour_le_piano_3_bachovich.mxl',
 'debussy_preludes_1_5_(c)galimberti.mxl',
 'debussy_preludes_1_6_(c)oguri.mxl',
 'debussy_preludes_1_8_(c)dery.mxl',
 'debussy_preludes_1_8_(c)galimberti.mxl',
 'debussy_preludes_1_9_(c)dery.mxl',
 'debussy_preludes_2_11_(c)galimberti.mxl',
 'debussy_suite_bergamasque_4_(c)galimberti.mxl',
 'scarlatti-d_esserciso_511_(c)icking-archive.mxl',
 'scarlatti-d_esserciso_517_(c)icking-archive.mxl',
 'scarlatti-d_esserciso_531_(c)icking-archive.mxl',
 'scarlatti_sonata_k-077_(c)sankey.mxl',
 'debussy_childrens_corner_1_(c)dery.mxl',
 'debussy_la_plus_que_lente_(c)lefeldt.mxl',
 'debussy_petit_negre_(c)yogore.mxl',
 'debussy_p

In [584]:
COMPOSERS1 = [DD_T,VD_T]
COMPOSERS2 = [DEBUSSY, VICTORIA]
benchmark_classifiers_twofeaturesets(COMPOSERS1, COMPOSERS2, (1,1), (1,2), CLASSIFIERS, VECTORIZER)

duration n-gram range: (1, 1) chord n-gram range: (1, 2)
LogisticRegression
[[199   0]
 [  1 332]]
F-measures
0.9981 (0.006) & 0.9979 (0.0067)
MLPClassifier
[[199   0]
 [  1 332]]
F-measures
0.9981 (0.006) & 0.9979 (0.0067)
MultinomialNB
[[199   0]
 [  6 327]]
F-measures
0.9887 (0.0097) & 0.988 (0.0104)
LinearSVC
[[199   0]
 [  1 332]]
F-measures
0.9981 (0.006) & 0.9979 (0.0067)
KNeighborsClassifier
[[199   0]
 [  0 333]]
F-measures
1.0 (0.0) & 1.0 (0.0)
-----


['victoria-da_laud-et_iesum_(c)alvarez.mxl',
 'victoria-da_laud-et_iesum_(c)alvarez.mxl',
 'victoria-da_falsobordon-4_(c)alvarez.mxl',
 'victoria-da_laud-et_iesum_(c)alvarez.mxl',
 'victoria-da_o_magnum_mysterium-6-agnus_dei_(c)alvarez.mxl',
 'victoria-da_victimae_paschale_(c)alvarez.mxl',
 'victoria-de_missa_ascendens_christus_3_(c)icking-archive.mxl',
 'victoria-de_o_magnum_mysterium_(brass)_(c)icking-archive.mxl',
 'victoria-da_laud-et_iesum_(c)alvarez.mxl']

In [585]:
COMPOSERS1 = [SD_T,VD_T]
COMPOSERS2 = [SCARLATTI, VICTORIA]
benchmark_classifiers_twofeaturesets(COMPOSERS1, COMPOSERS2, (1,1), (1,2), CLASSIFIERS, VECTORIZER)

duration n-gram range: (1, 1) chord n-gram range: (1, 2)
LogisticRegression
[[595   3]
 [  2 331]]
F-measures
0.9946 (0.0091) & 0.9944 (0.0095)
MLPClassifier
[[597   1]
 [  2 331]]
F-measures
0.9968 (0.0073) & 0.9966 (0.0077)
MultinomialNB
[[598   0]
 [  8 325]]
F-measures
0.9914 (0.0111) & 0.9905 (0.0123)
LinearSVC
[[598   0]
 [  1 332]]
F-measures
0.9989 (0.0034) & 0.9989 (0.0036)
KNeighborsClassifier
[[595   3]
 [  0 333]]
F-measures
0.9968 (0.0052) & 0.9965 (0.0056)
-----


['scarlatti-d_esserciso_335_(c)icking-archive.mxl',
 'scarlatti_sonata_k-041_(c)sankey.mxl',
 'scarlatti_sonata_k-308_(c)sankey.mxl',
 'victoria-da_laud-et_iesum_(c)alvarez.mxl',
 'victoria-da_victimae_paschale_(c)alvarez.mxl',
 'scarlatti_sonata_k-041_(c)sankey.mxl',
 'victoria-da_laud-et_iesum_(c)alvarez.mxl',
 'victoria-da_victimae_paschale_(c)alvarez.mxl',
 'victoria-da_falsobordon-4_(c)alvarez.mxl',
 'victoria-da_laud-et_iesum_(c)alvarez.mxl',
 'victoria-da_o_magnum_mysterium-6-agnus_dei_(c)alvarez.mxl',
 'victoria-da_victimae_paschale_(c)alvarez.mxl',
 'victoria-de_ave_maria_(brass)_(c)icking-archive.mxl',
 'victoria-de_missa_ascendens_christus_1_(c)icking-archive.mxl',
 'victoria-de_missa_ascendens_christus_3_(c)icking-archive.mxl',
 'victoria-de_o_magnum_mysterium_(brass)_(c)icking-archive.mxl',
 'victoria-da_laud-et_iesum_(c)alvarez.mxl',
 'scarlatti-d_esserciso_335_(c)icking-archive.mxl',
 'scarlatti_sonata_k-041_(c)sankey.mxl',
 'scarlatti_sonata_k-090_(c)sankey.mxl']

In [ ]:
# Identify the often-misclassified files, using both feature sets
# do the experiment 100 times with the best classifier, SVM
# then find pieces that are misclassified more than 50% of the time
COMPOSERS1 = [BD2_T,BD_T,DD_T,SD_T,VD_T]
COMPOSERS2 = [BACH, BEETHOVEN, DEBUSSY, SCARLATTI, VICTORIA]
CLASSIFIERS = [svm.LinearSVC(penalty='l2', C=5, loss='hinge')] # 
appendix = []
for i in range(100):
    appendix += benchmark_classifiers_twofeaturesets(COMPOSERS1, COMPOSERS2, (1,1), (1,2), CLASSIFIERS, VECTORIZER)
Counter(appendix).most_common()

duration n-gram range: (1, 1) chord n-gram range: (1, 2)
LinearSVC
[[2238    5    0    1    1]
 [  22  641    1    3    0]
 [   1    3  195    0    0]
 [   4   14    0  580    0]
 [   0    0    0    0  333]]
F-measures
0.9864 (0.005) & 0.9843 (0.0064)
-----
duration n-gram range: (1, 1) chord n-gram range: (1, 2)
LinearSVC
[[2238    5    0    1    1]
 [  23  640    2    2    0]
 [   2    3  194    0    0]
 [   4   14    0  580    0]
 [   0    0    0    0  333]]
F-measures
0.9859 (0.0078) & 0.9832 (0.0096)
-----
duration n-gram range: (1, 1) chord n-gram range: (1, 2)
LinearSVC
[[2238    6    0    1    0]
 [  22  642    1    2    0]
 [   1    3  195    0    0]
 [   6   13    0  579    0]
 [   0    0    0    0  333]]
F-measures
0.9864 (0.0059) & 0.9848 (0.0051)
-----
duration n-gram range: (1, 1) chord n-gram range: (1, 2)
LinearSVC
[[2237    6    0    2    0]
 [  22  640    1    4    0]
 [   1    2  195    1    0]
 [   4   14    0  580    0]
 [   0    0    0    0  333]]
F-measures
0.985